In [ ]:
from pathlib import Path
import json
import os
import numpy as np
from tqdm import tqdm
from lac.params import IMG_WIDTH, IMG_HEIGHT, FL_X, FL_Y
from lac.utils.frames import cam_to_world

data_path = Path("/home/shared/data_raw/LAC/runs/full_spiral_map1_preset1_recovery_agent")

with open(data_path / "data_log.json", "r") as f:
    data_log = json.load(f)

cam_names = ["FrontLeft", "Left", "Right"]

transforms = {
    "w": IMG_WIDTH,
    "h": IMG_HEIGHT,
    "fl_x": FL_X,
    "fl_y": FL_Y,
    "cx": IMG_WIDTH / 2,
    "cy": IMG_HEIGHT / 2,
    "k1": 0.0,
    "k2": 0.0,
    "p1": 0.0,
    "p2": 0.0,
    "ply_file_path": "dem_points.ply",
    "frames": [],
}

lims = (80, np.nan)
step = 8 * 4
for i in tqdm(range(len(data_log["frames"]))):
    if i < lims[0] or i % step != 0 or i > lims[1]:
        continue
    pose = np.array(data_log["frames"][i]["pose"])
    for cam_name in cam_names:
        ocv_T_world = cam_to_world(pose, cam_name)
        # OpenCV: Z-forward, Y-down, X-right
        # OpenGL: Z-back, Y-up, X-right
        ogl_T_ocv = np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
        ogl_T_world = ogl_T_ocv @ ocv_T_world
        transforms["frames"].append(
            {"file_path": f"{cam_name}/{i:06d}.png", "transform_matrix": np.linalg.inv(ogl_T_world).tolist()}
        )

with open(data_path / "transforms.json", "w") as f:
    json.dump(transforms, f, indent=4)

In [ ]:
def generate_ply_from_points(points: np.ndarray, colors: np.ndarray, filename: str = "pc.ply") -> None:
    """Generate PLY file from points for gaussian splatting initialization"""
    # Check that both points and colors are Nx3 arrays
    assert points.shape[1] == 3, "Points array should have shape Nx3"
    assert colors.shape[1] == 3, "Colors array should have shape Nx3"
    assert points.shape[0] == colors.shape[0], "Points and colors arrays must have the same number of rows"

    num_vertices = points.shape[0]

    # Open file for writing
    with open(filename, "w") as ply_file:
        # Write PLY header
        ply_file.write("ply\n")
        ply_file.write("format ascii 1.0\n")
        ply_file.write(f"element vertex {num_vertices}\n")
        ply_file.write("property float x\n")
        ply_file.write("property float y\n")
        ply_file.write("property float z\n")
        ply_file.write("property uint8 red\n")
        ply_file.write("property uint8 green\n")
        ply_file.write("property uint8 blue\n")
        ply_file.write("end_header\n")

        # Write vertex data (points and colors)
        for point, color in zip(points, colors):
            x, y, z = point
            r, g, b = color.astype(int)  # Convert color to int if not already
            ply_file.write(f"{x} {y} {z} {r} {g} {b}\n")

In [ ]:
map_gt = np.load(
    "/home/shared/data_raw/LAC/heightmaps/competition/Moon_Map_01_preset_1.dat",
    allow_pickle=True,
)

# Map gt is NxMx4, generate points (NxM/3)x3 from the x,y,z coordinates subsampling
ds = 1
points = map_gt[:, :, :3].reshape(-1, 3)[::ds]
colors = np.full_like(points, 255 / 2.0)
generate_ply_from_points(points, colors, data_path / "dem_points.ply")

In [ ]:
from lac.utils.plotting import plot_poses
import plotly.graph_objects as go

fig = go.Figure()
pose = np.array(data_log["frames"][80]["pose"])
cam_names = ["FrontLeft", "Left", "Right", "FrontRight"]
for cam_name in cam_names:
    ocv_T_world = cam_to_world(pose, cam_name)
    ogl_T_world = ogl_T_ocv @ ocv_T_world
    world_T_ogl = np.linalg.inv(ogl_T_world)
    world_T_ocv = np.linalg.inv(ocv_T_world)
    plot_poses([world_T_ocv], fig, line_width=10)
fig.update_layout(scene_aspectmode="data", width=800, height=400)
fig.show()